In [65]:
import keras
import numpy as np
import cv2
import random
from keras.datasets import mnist
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Flatten, Dense, Dropout,Flatten, Lambda,Concatenate
from keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.preprocessing import image
from keras.layers import Dropout
from keras.saving import register_keras_serializable
from tensorflow.keras.callbacks import LambdaCallback
from skimage.filters import gabor_kernel
from scipy.ndimage import convolve# from keras import backend as K
import tensorflow.keras.backend as K
import os
import tensorflow as tf

In [66]:
def create_base_net(input_shape):
  
# Define input layer
    inputs = Input(shape=input_shape)

    # Convolutional layer 1
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', name='conv1')(inputs)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 2
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', name='conv2')(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 3
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv3')(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 4
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv4')(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten the output for fully connected layers
    x = Flatten()(x)

    # Fully Connected layer 1
    x = Dense(1000, activation='relu', name='fc1')(x)

    # Fully Connected layer 2
    fc2_output = Dense(128, activation='sigmoid', name='fc2')(x)

    # Output layer (features from FC layer 2)
    features = fc2_output

    # Create model
    model = Model(inputs=inputs, outputs=features)

    # Summary of the model
    # model.summary()
  
    return model

@register_keras_serializable()
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    y_true_int = tf.cast(y_true, tf.float32)

    return K.mean(y_true_int * square_pred + (1 - y_true_int) * margin_square)

@register_keras_serializable()
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [67]:
input_shape=(250,250,1)

Train and validation dataset preprocessing

In [ ]:
# Input and output directories
input_directory = '/kaggle/input/train-test/d/train'
output_directory = '/kaggle/working/train'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

def gabor_filter(ksize, sigma, theta, lambd, gamma):
    phi = 0.5 * np.pi
    kernel = np.zeros((ksize, ksize), dtype=np.float32)
    for x in range(-ksize // 2 + 1, ksize // 2 + 1):
        for y in range(-ksize // 2 + 1, ksize // 2 + 1):
            x_prime = x * np.cos(theta) + y * np.sin(theta)
            y_prime = -x * np.sin(theta) + y * np.cos(theta)
            kernel[x + ksize // 2, y + ksize // 2] = np.exp(-0.5 * (x_prime ** 2 + (gamma * y_prime) ** 2) / (sigma ** 2)) * np.cos(2 * np.pi * x_prime / lambd + phi)
    return kernel

# Define Gabor filter parameters
ksize = 31  # Kernel size
sigma = 7  # Standard deviation of the Gaussian envelope
theta = np.pi / 4  # Orientation of the normal to the parallel stripes of a Gabor function
lambd = 10 # Wavelength of the sinusoidal factor
gamma = 0.5  # Spatial aspect ratio     

# Iterate over the files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.bmp'):

         # Read the image
        imge = cv2.imread(os.path.join(input_directory, filename), cv2.IMREAD_GRAYSCALE)

        # Define the region of interest (ROI) to cut off the right part
        start_col = 0  # Start column index
        end_col = 670    # End column index
        # Crop the image
        cropped_image = imge[:, start_col:end_col]

        # Apply Otsu's thresholding for segmentation
        val, binary_img = cv2.threshold(cropped_image, 0, 255, cv2.THRESH_BINARY+ cv2.THRESH_OTSU)

        # Find contours
        contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a mask for the largest contour
        mask = cv2.drawContours(
            np.zeros_like(binary_img),     # Create a blank image of the same size as the binary image
            [largest_contour],             # Pass the largest contour as a list
            contourIdx=-1,                 # Draw all contours
            color=255,                     # Contour color (white)
            thickness=cv2.FILLED           # Fill the contour
        )

        # Apply the mask to the original image
        binary_img_1 = cv2.bitwise_and(binary_img, mask) #only keeping the palm part of image

        # Find contours
        contours, _ = cv2.findContours(binary_img_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the largest contour (palm contour)
        palm_contour = max(contours, key=cv2.contourArea)

        # Calculate the distance transform
        dist_transform = cv2.distanceTransform(binary_img_1, cv2.DIST_L2, 5)

        # Find the center and radius of the maximum inscribed circle
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(dist_transform)
        center = max_loc
        radius = int(max_val)

        # Draw the maximum inscribed circle on the original image
        result = binary_img_1.copy()
        cv2.circle(result, center, radius, (0, 0, 255), 2)
        
        # Calculate the side length of the square
        side_length = int(np.sqrt(2) * radius)

        # Calculate the top-left corner of the square
        top_left = (center[0] - side_length // 2, center[1] - side_length // 2)
        target_size = (128, 128)
        resized_square = cv2.resize(result[top_left[1]:top_left[1]+side_length, top_left[0]:top_left[0]+side_length], target_size)

        # Draw the square on the original image
        result = result.copy()
        cv2.rectangle(result, top_left, (top_left[0] + side_length, top_left[1] + side_length), (0, 0, 255), 2)
        
        # Calculate the top-left corner coordinates of the square
        x = center[0] - side_length // 2
        y = center[1] - side_length // 2

        # Extract the square portion from the original image
        square_portion = imge[y:y+side_length, x:x+side_length]
       
        # Create Gabor filter kernel
        gabor_kernel = gabor_filter(ksize, sigma, theta, lambd, gamma)

        # Apply the Gabor filter to the image
        filtered_image = convolve(square_portion.astype(float), gabor_kernel)

        # Normalize filtered image to range [0, 255]
        filtered_image = (filtered_image - filtered_image.min()) / (filtered_image.max() - filtered_image.min()) * 255

        # Convert to uint8
        filtered_image = filtered_image.astype(np.uint8)
 
        # Save the filtered image to the output directory
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path,filtered_image)

Test dataset preprocessing

In [ ]:
# Input and output directories
input_directory = '/kaggle/input/train-test/d/test'
output_directory = '/kaggle/working/test'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

def gabor_filter(ksize, sigma, theta, lambd, gamma):
    phi = 0.5 * np.pi
    kernel = np.zeros((ksize, ksize), dtype=np.float32)
    for x in range(-ksize // 2 + 1, ksize // 2 + 1):
        for y in range(-ksize // 2 + 1, ksize // 2 + 1):
            x_prime = x * np.cos(theta) + y * np.sin(theta)
            y_prime = -x * np.sin(theta) + y * np.cos(theta)
            kernel[x + ksize // 2, y + ksize // 2] = np.exp(-0.5 * (x_prime ** 2 + (gamma * y_prime) ** 2) / (sigma ** 2)) * np.cos(2 * np.pi * x_prime / lambd + phi)
    return kernel

# Define Gabor filter parameters
ksize = 31  # Kernel size
sigma = 7  # Standard deviation of the Gaussian envelope
theta = np.pi / 4  # Orientation of the normal to the parallel stripes of a Gabor function
lambd = 10 # Wavelength of the sinusoidal factor
gamma = 0.5  # Spatial aspect ratio     


# Iterate over the files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.bmp'):

         # Read the image
        imge = cv2.imread(os.path.join(input_directory, filename), cv2.IMREAD_GRAYSCALE)

        # Define the region of interest (ROI) to cut off the right part
        start_col = 0  # Start column index
        end_col = 670    # End column index
        # Crop the image
        cropped_image = imge[:, start_col:end_col]

        # Apply Otsu's thresholding for segmentation
        val, binary_img = cv2.threshold(cropped_image, 0, 255, cv2.THRESH_BINARY+ cv2.THRESH_OTSU)

        # Find contours
        contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Create a mask for the largest contour
        mask = cv2.drawContours(
            np.zeros_like(binary_img),     # Create a blank image of the same size as the binary image
            [largest_contour],             # Pass the largest contour as a list
            contourIdx=-1,                 # Draw all contours
            color=255,                     # Contour color (white)
            thickness=cv2.FILLED           # Fill the contour
        )

        # Apply the mask to the original image
        binary_img_1 = cv2.bitwise_and(binary_img, mask) #only keeping the palm part of image

        # Find contours
        contours, _ = cv2.findContours(binary_img_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the largest contour (palm contour)
        palm_contour = max(contours, key=cv2.contourArea)

        # Calculate the distance transform
        dist_transform = cv2.distanceTransform(binary_img_1, cv2.DIST_L2, 5)

        # Find the center and radius of the maximum inscribed circle
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(dist_transform)
        center = max_loc
        radius = int(max_val)

        # Draw the maximum inscribed circle on the original image
        result = binary_img_1.copy()
        cv2.circle(result, center, radius, (0, 0, 255), 2)
        
        # Calculate the side length of the square
        side_length = int(np.sqrt(2) * radius)

        # Calculate the top-left corner of the square
        top_left = (center[0] - side_length // 2, center[1] - side_length // 2)
        target_size = (128, 128)
        resized_square = cv2.resize(result[top_left[1]:top_left[1]+side_length, top_left[0]:top_left[0]+side_length], target_size)

        # Draw the square on the original image
        result = result.copy()
        cv2.rectangle(result, top_left, (top_left[0] + side_length, top_left[1] + side_length), (0, 0, 255), 2)
        
        # Calculate the top-left corner coordinates of the square
        x = center[0] - side_length // 2
        y = center[1] - side_length // 2

        # Extract the square portion from the original image
        square_portion = imge[y:y+side_length, x:x+side_length]
       
        # Create Gabor filter kernel
        gabor_kernel = gabor_filter(ksize, sigma, theta, lambd, gamma)

        # Apply the Gabor filter to the image
        filtered_image = convolve(square_portion.astype(float), gabor_kernel)

        # Normalize filtered image to range [0, 255]
        filtered_image = (filtered_image - filtered_image.min()) / (filtered_image.max() - filtered_image.min()) * 255

        # Convert to uint8
        filtered_image = filtered_image.astype(np.uint8)

        # Save the filtered image to the output directory
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path,filtered_image )

Making pairs of 4 for the model

In [68]:
#directory = "/kaggle/input/roi-extracted-dataset/jicrop/train"
directory = "/kaggle/input/roi-gabor/roi_gabor/train"

filenames = os.listdir(directory)

# Initialize an empty list to store split filenames
split_filenames = []
X1=[]
XX1=[]
y=[]
vy=[]
hand={}
lhand={}
rhand={}
# Split each filename by underscore and append to the list
for filename in filenames:
    split_filename = filename.split('_')

    # Rename files by removing 850(Which represents wavelength)
    # split_filename.pop(2)
    # new_filename = "_".join(split_filename)
    # os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
    
    
    #Assigning each person to their hands in the dictionary.
    if split_filename[0] in hand:
        hand[split_filename[0]].append(filename)
        if split_filename[1]=='l':
            if split_filename[0] in lhand:
                lhand[split_filename[0]].append(filename)
            else:
                lhand[split_filename[0]]=[filename]
        else:
            if split_filename[0] in rhand:
                rhand[split_filename[0]].append(filename)
            else:
                rhand[split_filename[0]]=[filename]
            # rhand[split_filename[0]].append(filename)
    else:
        hand[split_filename[0]]=[filename]
        if split_filename[1]=='l':
            lhand[split_filename[0]]=[filename]
        else:
            rhand[split_filename[0]]=[filename]
    
    split_filenames.append(split_filename)

# print(lhand.values())
# print(hand)
#print(lhand)
# print(rhand)
count=0
#Training pairs
for i in range(13):
    
    l1k=list(lhand.keys())[i]   #Choosing a person
    l1=list(lhand.values())[i]  #Getting that person's left hands
    r1=list(rhand[l1k])         #Getting that person's right hands
#     print(l1k)
    
    print(l1)
#     print(r1)
    for count in range(len(l1)):
        if count<=2:#Different person
            ran=random.randint(30,60)
            l2k=list(lhand.keys())[i+ran]
            l2=list(lhand.values())[i+ran]
            r2=list(rhand[l2k])
            X1.append([l1[count],r1[count],l2[count],r2[count]])
            y.append(0)
        else:#Same person
            r=random.sample(range(0,6),2)
            #print(r)
            X1.append([l1[r[0]],r1[r[0]],l1[r[1]],r1[r[1]]])
            y.append(1)

#validation pairs
for i in range(18,22):
    
    l1k=list(lhand.keys())[i] 
    l1=list(lhand.values())[i]
    r1=list(rhand[l1k])
#     print(l1k)
#     print(l1)
#     print(r1)
    for count in range(len(l1)):
        if count<=2: #Different person
            ran=random.randint(30,60)
            l2k=list(lhand.keys())[i+ran]
            l2=list(lhand.values())[i+ran]
            r2=list(rhand[l2k])
            if ([l1[count],r1[count],l2[count],r2[count]]) in X1:
                continue
            XX1.append([l1[count],r1[count],l2[count],r2[count]])
            vy.append(0)
        else:   #Same person
            r=random.sample(range(0,6),2)
            #print(r)
            XX1.append([l1[r[0]],r1[r[0]],l1[r[1]],r1[r[1]]])
            vy.append(1)
# print(split_filenames)
print(np.shape(X1))
print(X1)
print(y)
print(XX1)
print(vy)
#print(y)

['009_l_01.jpg', '009_l_02.jpg', '009_l_04.jpg', '009_l_06.jpg', '009_l_05.jpg', '009_l_03.jpg']
['016_l_04.jpg', '016_l_01.jpg', '016_l_03.jpg', '016_l_05.jpg', '016_l_02.jpg', '016_l_06.jpg']
['034_l_03.jpg', '034_l_01.jpg', '034_l_05.jpg', '034_l_06.jpg', '034_l_04.jpg', '034_l_02.jpg']
['026_l_05.jpg', '026_l_06.jpg', '026_l_01.jpg', '026_l_04.jpg', '026_l_02.jpg', '026_l_03.jpg']
['052_l_05.jpg', '052_l_03.jpg', '052_l_01.jpg', '052_l_02.jpg', '052_l_06.jpg', '052_l_04.jpg']
['078_l_01.jpg', '078_l_06.jpg', '078_l_03.jpg', '078_l_04.jpg', '078_l_05.jpg', '078_l_02.jpg']
['076_l_03.jpg', '076_l_04.jpg', '076_l_06.jpg', '076_l_05.jpg', '076_l_02.jpg', '076_l_01.jpg']
['023_l_04.jpg', '023_l_03.jpg', '023_l_06.jpg', '023_l_02.jpg', '023_l_01.jpg', '023_l_05.jpg']
['007_l_03.jpg', '007_l_01.jpg', '007_l_05.jpg', '007_l_04.jpg', '007_l_06.jpg', '007_l_02.jpg']
['071_l_03.jpg', '071_l_05.jpg', '071_l_02.jpg', '071_l_01.jpg', '071_l_06.jpg', '071_l_04.jpg']
['041_l_05.jpg', '041_l_04.jpg

Pairs for testing

In [69]:
directory = "/kaggle/input/roi-gabor/roi_gabor/test"
filenames = os.listdir(directory)

# Initialize an empty list to store split filenames
split_filenames = []

T_X=[]
T_y=[]
hand={}
lhand={}
rhand={}
for filename in filenames:
    split_filename = filename.split('_')

    # Rename files by removing 850(Which represents wavelength)
    # split_filename.pop(2)
    # new_filename = "_".join(split_filename)
    # os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
    if split_filename[0] in hand:
        hand[split_filename[0]].append(filename)
        if split_filename[1]=='l':
            if split_filename[0] in lhand:
                lhand[split_filename[0]].append(filename)
            else:
                lhand[split_filename[0]]=[filename]
        else:
            if split_filename[0] in rhand:
                rhand[split_filename[0]].append(filename)
            else:
                rhand[split_filename[0]]=[filename]
    else:
        hand[split_filename[0]]=[filename]
        if split_filename[1]=='l':
            lhand[split_filename[0]]=[filename]
        else:
            rhand[split_filename[0]]=[filename]
    
    split_filenames.append(split_filename)

count=0
for i in range(2):
    
    l1k=list(lhand.keys())[i]
    l1=list(lhand.values())[i]
    r1=list(rhand[l1k])
#     print(l1k)
#     print(l1)
#     print(r1)
    for count in range(len(l1)):
        if count<=2:
            ran=random.randint(1,6)
            l2k=list(lhand.keys())[i+ran]
            l2=list(lhand.values())[i+ran]
            r2=list(rhand[l2k])
            T_X.append([l1[count],r1[count],l2[count],r2[count]])
            T_y.append(0)
        else:
            r=random.sample(range(0,6),2)
            #print(r)
            T_X.append([l1[r[0]],r1[r[0]],l1[r[1]],r1[r[1]]])
            T_y.append(1)
print(T_X)
print(T_y)

[['098_l_04.jpg', '098_r_01.jpg', '084_l_03.jpg', '084_r_05.jpg'], ['098_l_01.jpg', '098_r_02.jpg', '090_l_04.jpg', '090_r_02.jpg'], ['098_l_02.jpg', '098_r_05.jpg', '100_l_02.jpg', '100_r_01.jpg'], ['098_l_03.jpg', '098_r_06.jpg', '098_l_02.jpg', '098_r_05.jpg'], ['098_l_02.jpg', '098_r_05.jpg', '098_l_03.jpg', '098_r_06.jpg'], ['098_l_04.jpg', '098_r_01.jpg', '098_l_02.jpg', '098_r_05.jpg'], ['084_l_03.jpg', '084_r_05.jpg', '091_l_01.jpg', '091_r_05.jpg'], ['084_l_06.jpg', '084_r_01.jpg', '096_l_05.jpg', '096_r_06.jpg'], ['084_l_01.jpg', '084_r_06.jpg', '091_l_04.jpg', '091_r_02.jpg'], ['084_l_04.jpg', '084_r_04.jpg', '084_l_05.jpg', '084_r_02.jpg'], ['084_l_01.jpg', '084_r_06.jpg', '084_l_03.jpg', '084_r_05.jpg'], ['084_l_03.jpg', '084_r_05.jpg', '084_l_04.jpg', '084_r_04.jpg']]
[0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]


Getting each image's filenames and storing it as array of pixel values in X2...X5 for model's computation(Training,X21...X51-validation,T_X2=testing)

In [70]:
train_image = []
X2=[]
X3=[]
X4=[]
X5=[]
X21=[]
X31=[]
X41=[]
X51=[]
T_X2=[]
T_X3=[]
T_X4=[]
T_X5=[]

for i in X1:
    train_subset=[]
    c=0
    for j in i:
        # print(c)
        img = image.load_img('/kaggle/input/roi-gabor/roi_gabor/train/'+j, target_size=(250,250), color_mode="grayscale")
        img = image.img_to_array(img)
        img = img/255
        train_subset.append(img)
        if c==0:
            X2.append(img)
        elif c==1:
            X3.append(img)
        elif c==2:
            X4.append(img)
        elif c==3:
            X5.append(img)
        c+=1

for i in XX1:
    train_subset=[]
    c=0
    for j in i:
        # print(c)
        img = image.load_img('/kaggle/input/roi-gabor/roi_gabor/train/'+j, target_size=(250,250), color_mode="grayscale")
        img = image.img_to_array(img)
        img = img/255
        train_subset.append(img)
        if c==0:
            X21.append(img)
        elif c==1:
            X31.append(img)
        elif c==2:
            X41.append(img)
        elif c==3:
            X51.append(img)
        c+=1
for i in T_X:
    train_subset=[]
    c=0
    for j in i:
        # print(c)
        img = image.load_img('/kaggle/input/roi-gabor/roi_gabor/test/'+j, target_size=(250,250), color_mode="grayscale")
        img = image.img_to_array(img)
        img = img/255
        train_subset.append(img)
        if c==0:
            T_X2.append(img)
        elif c==1:
            T_X3.append(img)
        elif c==2:
            T_X4.append(img)
        elif c==3:
            T_X5.append(img)
        c+=1
    
X2=np.array(X2)
X3=np.array(X3)
X4=np.array(X4)
X5=np.array(X5)
y=np.array(y)
X21=np.array(X21)
X31=np.array(X31)
X41=np.array(X41)
X51=np.array(X51)
vy=np.array(vy)
T_X2=np.array(T_X2)
T_X3=np.array(T_X3)
T_X4=np.array(T_X4)
T_X5=np.array(T_X5)
T_y=np.array(T_y)
print(np.shape(X2))
print(type(X2))
print(np.shape(X21))
print(type(X21))

(78, 250, 250, 1)
<class 'numpy.ndarray'>
(24, 250, 250, 1)
<class 'numpy.ndarray'>


Making the final model from the base CNN model

In [71]:
base_network = create_base_net((250,250,1))
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
input_c = Input(shape=input_shape)
input_d = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)
concatenated_features = Concatenate()([processed_a, processed_b])

processed_c = base_network(input_c)
processed_d = base_network(input_d)
concatenated_outputs = Concatenate()([processed_c, processed_d])

difference = Lambda(lambda x: abs(x[0] - x[1]), name='difference')([concatenated_features, concatenated_outputs])

probability = Dense(1, activation='sigmoid', name='probability')(difference)

model = Model([input_a, input_b,input_c,input_d], outputs=probability)

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_7        │ (None, 128)       │ 14,641,576 │ input_layer_7[0]… │
│ (Functional)        │                   │            │ input_layer_8[0]… │
│                     │                   │            │ input_layer_9[0]… │
│                     │                   │            │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 256)       │          0 │ functional_7[0][… │
│ (Concatenate)       │                   │            │ functional_7[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 256)       │          0 │ functional_7[2][… │
│ (Concatenate)       │                   │            │ functional_7[3][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ difference (Lambda) │ (None, 256)       │          0 │ concatenate_2[0]… │
│                     │                   │            │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ probability (Dense) │ (None, 1)         │        257 │ difference[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,641,833 (55.85 MB)

 Trainable params: 14,641,321 (55.85 MB)

 Non-trainable params: 512 (2.00 KB)

Training

In [ ]:
adam = Adam(learning_rate=0.0001)#0.0001
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
model.fit([X2,X3,X4,X5], y,batch_size=32,epochs=300,validation_data=([X21,X31,X41,X51],vy),)

In [73]:
res=model.predict([T_X2,T_X3,T_X4,T_X5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


W0000 00:00:1715786014.864979     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [74]:
res=(res>0.5).astype('float32') #assigning class based on probability value
print(res)
print(T_y)

[[1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
[0 0 0 1 1 1 0 0 0 1 1 1]


In [75]:
#predicting accuracy on training dataset
total=len(res)
corr=0
for i in range(len(res)):
    if res[i]==T_y[i]:
        corr+=1
accu=corr/total
print(accu)

0.6666666666666666


In [76]:
model.save('jawa.keras')

In [ ]:
model2=load_model('/kaggle/input/palmveinmodel/keras/finalversion/1/jawa.keras',safe_mode=False,custom_objects={'contrastive_loss': contrastive_loss,'accuracy':accuracy})

In [ ]:
# model2.summary()

In [ ]:
# adam = Adam(learning_rate=0.001)#0.0001
# model2.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
# model2.fit([X2,X3,X4,X5], y,batch_size=128,epochs=300,validation_data=([X21,X31,X41,X51],vy),)
# #           callbacks=[LambdaCallback(on_epoch_end=print_weight_difference)])
#         #   validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y)

In [61]:
# res=model2.predict([T_X2,T_X3,T_X4,T_X5])
# res=(res>0.5).astype('float32')
# print(res)
# print(T_y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[0 0 0 1 1 1 0 0 0 1 1 1]


In [77]:
total = len(res)
corr=0
for i in range(len(res)):
    if res[i]==T_y[i]:
        corr+=1
accu=corr/total
print(accu)
true_positives = sum(1 for i in range(total) if res[i] == T_y[i] and res[i] == 1)
false_positives = sum(1 for i in range(total) if res[i] != T_y[i] and res[i] == 1)
false_negatives = sum(1 for i in range(total) if res[i] != T_y[i] and res[i] == 0)

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

print("Precision:", precision)
print("Recall:", recall)


0.6666666666666666
Precision: 0.6
Recall: 1.0


In [78]:
T_X

[['098_l_04.jpg', '098_r_01.jpg', '084_l_03.jpg', '084_r_05.jpg'],
 ['098_l_01.jpg', '098_r_02.jpg', '090_l_04.jpg', '090_r_02.jpg'],
 ['098_l_02.jpg', '098_r_05.jpg', '100_l_02.jpg', '100_r_01.jpg'],
 ['098_l_03.jpg', '098_r_06.jpg', '098_l_02.jpg', '098_r_05.jpg'],
 ['098_l_02.jpg', '098_r_05.jpg', '098_l_03.jpg', '098_r_06.jpg'],
 ['098_l_04.jpg', '098_r_01.jpg', '098_l_02.jpg', '098_r_05.jpg'],
 ['084_l_03.jpg', '084_r_05.jpg', '091_l_01.jpg', '091_r_05.jpg'],
 ['084_l_06.jpg', '084_r_01.jpg', '096_l_05.jpg', '096_r_06.jpg'],
 ['084_l_01.jpg', '084_r_06.jpg', '091_l_04.jpg', '091_r_02.jpg'],
 ['084_l_04.jpg', '084_r_04.jpg', '084_l_05.jpg', '084_r_02.jpg'],
 ['084_l_01.jpg', '084_r_06.jpg', '084_l_03.jpg', '084_r_05.jpg'],
 ['084_l_03.jpg', '084_r_05.jpg', '084_l_04.jpg', '084_r_04.jpg']]

In [79]:
T_y

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1])

In [80]:
res

array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)